In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


/Users/aayeshaaayesha/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import pandas as pd
df_profile1_sample = pd.read_excel('buyer_profile1_hobby_sample.xlsx')
df_profile2_sample = pd.read_excel('buyer_profile2_hobby_sample.xlsx')
df_profile3_sample = pd.read_excel('buyer_profile3_hobby_sample.xlsx')
df_profile4_sample = pd.read_excel('buyer_profile4_hobby_sample.xlsx')
df_profile5_sample = pd.read_excel('buyer_profile5_hobby_sample.xlsx')
df_profile5_sample

,Unnamed: 0.1,Unnamed: 0,UID,Name,interface1b,interface2b,interface3b,function1b,function2b,accomp1b,accomp2b
0,6,74,136,Joan,Yes,"Yes, but it would have been good to have the i...",It was easy to understand,"Yes, it would be easy if they have Telegram",Yes. A recommendation without the item informa...,"Could be useful, yes. But sharing the link to ...",Yes
1,0,13,87,Manuel,"Yes, gives a quick overview","Yes, it was intuitive",Very easy to understand,"Yes, after a few tries, it was easy to use","Yes, helps when ranking the items","Yes, otherwise they would not be able to recei...","Yes, cloth images are indeed helpful"
2,5,72,137,Roman,"yes, but it could be better if you don't have ...","relatively, yes","yes, seems so","yes, it was shared without any problems","yes, though it's not the only criteria, which ...","relatively useful, though the choice of items ...",yes
3,2,41,142,Luka,Yes because it shows the preferred items.,First it was a little bit confusing but with t...,Without the youtube video it wouldn't be easy ...,Yes!,"Yes some of them do eg. sports, so the feedbac...","It depends, not all of the group members have ...","Yes the clothing images are really good, else ..."
4,4,47,129,Dániel Szakács,It's easy to add or remove items from the wish...,We used Telegram and it was very straight forw...,"Easy to understand, however when there are mul...",Yes because most of my friends use either Tele...,"Yes, as it determines the style of the clothin...","Yes, because for clothing recommendations I mo...","Yes, I think it the visual aspects are the mos..."
5,1,29,100,Kasra,yes it was a good overview,no,yes,no it was too much spam in the bot,yes,yes,yes
6,3,45,140,Christoph,The rating on a computer is little tedious as ...,I am not sure if I made I correctly. Maybe you...,I don't think I looked at the information too ...,"As mentioned above. In principle yes, but I ac...","Yes, absolutely. Ranking is far easier if you ...",In general I can see that people use recommend...,Yes absolutely. I don't think you need any mor...


In [5]:
from statistics import mean 
col_list = ['accomp1b', 'accomp2b']
def sentiForCol(col_list):
    pos_mean_ls = []
    for k in range(df_profile1_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_profile1_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol(col_list))
profile1 = sentiForCol(col_list)

[0.61158335, 0.92426664, 0.25089774, 0.36381948, 0.54198956, 0.25107777, 0.25500366]


In [6]:
col_list = ['accomp1b', 'accomp2b']
def sentiForCol1(col_list):
    pos_mean_ls = []
    for k in range(df_profile2_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_profile2_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol1(col_list))
profile2 = sentiForCol1(col_list)

[0.33198825, 0.09593158, 0.02250006, 0.2382182, 0.36381948, 0.21194611, 0.54198956]


In [7]:
col_list = ['accomp1b', 'accomp2b']
def sentiForCol1(col_list):
    pos_mean_ls = []
    for k in range(df_profile3_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_profile3_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol1(col_list))
profile3 = sentiForCol1(col_list)

[0.36381948, 0.47874522, 0.24331254, 0.42992923, 0.02250006, 0.61158335, 0.14821877]


In [8]:
col_list = ['accomp1b', 'accomp2b']
def sentiForCol1(col_list):
    pos_mean_ls = []
    for k in range(df_profile4_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_profile4_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol1(col_list))
profile4 = sentiForCol1(col_list)

[0.47812977, 0.22022772, 0.21194611, 0.6354259, 0.25500366, 0.36381948, 0.25089774]


In [9]:
col_list = ['accomp1b', 'accomp2b']
def sentiForCol1(col_list):
    pos_mean_ls = []
    for k in range(df_profile5_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_profile5_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol1(col_list))
profile5 = sentiForCol1(col_list)

[0.6354259, 0.47874522, 0.61158335, 0.47572064, 0.33198825, 0.48006797, 0.22022772]


In [10]:
df_profile1_interface_t_test = pd.DataFrame({'profile': 'profile1', 'interface_pos': profile1 })
df_profile2_interface_t_test = pd.DataFrame({'profile': 'profile2', 'interface_pos': profile2 })
df_profile3_interface_t_test = pd.DataFrame({'profile': 'profile3', 'interface_pos': profile3 })
df_profile4_interface_t_test = pd.DataFrame({'profile': 'profile4', 'interface_pos': profile4 })
df_profile5_interface_t_test = pd.DataFrame({'profile': 'profile5', 'interface_pos': profile5 })


In [11]:
df_profiles_combine_interface_t_test = pd.concat([df_profile1_interface_t_test, df_profile2_interface_t_test, df_profile3_interface_t_test, df_profile4_interface_t_test, df_profile5_interface_t_test], axis=0)
#df_hobby_combine_interface_t_test = pd.concat([df_arts_interface_t_test, df_entertain_interface_t_test, df_study_interface_t_test, df_nature_interface_t_test], axis=0)
df_profiles_combine_interface_t_test

,profile,interface_pos
0,profile1,0.611583
1,profile1,0.924267
2,profile1,0.250898
3,profile1,0.363819
4,profile1,0.541990
5,profile1,0.251078
6,profile1,0.255004
0,profile2,0.331988
1,profile2,0.095932
2,profile2,0.022500


In [11]:
from scipy import stats
import numpy as np


interface_pos_profile1 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == 'profile1']['interface_pos']
interface_pos_profile2 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == 'profile2']['interface_pos']
interface_pos_profile3 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == 'profile3']['interface_pos']
interface_pos_profile4 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == 'profile4']['interface_pos']
interface_pos_profile5 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == 'profile5']['interface_pos']

# Perform the independent t-test
t_stat, p_value = stats.wilcoxon(interface_pos_profile1, interface_pos_profile5)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis - there is a significant difference in interface experience between age groups.")
else:
    print("Fail to reject the null hypothesis - there is no significant difference in interface experience between age groups.")

T-statistic: 13.0
P-value: 0.9375
Fail to reject the null hypothesis - there is no significant difference in interface experience between age groups.


In [12]:
from scipy import stats
from itertools import combinations
hobbies = df_profiles_combine_interface_t_test['profile'].unique()

# Perform pairwise t-tests
pairwise_results = []
for (hobby1, hobby2) in combinations(hobbies, 2):
    group1 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == hobby1]['interface_pos']
    group2 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == hobby2]['interface_pos']
    t_stat, p_value = stats.ttest_ind(group1, group2)
    pairwise_results.append(((hobby1, hobby2), t_stat, p_value))

# Print results
for (hobby1, hobby2), t_stat, p_value in pairwise_results:
    print(f"{hobby1} vs. {hobby2}: T-statistic = {t_stat:.2f}, P-value = {p_value:.4f}")


profile1 vs. profile2: T-statistic = 1.71, P-value = 0.1122
profile1 vs. profile3: T-statistic = 1.05, P-value = 0.3148
profile1 vs. profile4: T-statistic = 0.99, P-value = 0.3417
profile1 vs. profile5: T-statistic = -0.05, P-value = 0.9645
profile2 vs. profile3: T-statistic = -0.69, P-value = 0.5004
profile2 vs. profile4: T-statistic = -0.98, P-value = 0.3486
profile2 vs. profile5: T-statistic = -2.37, P-value = 0.0352
profile3 vs. profile4: T-statistic = -0.17, P-value = 0.8665
profile3 vs. profile5: T-statistic = -1.41, P-value = 0.1833
profile4 vs. profile5: T-statistic = -1.43, P-value = 0.1786
